In [1]:
import factory

from django.utils import timezone

from wagtail.core.models import Page

In [2]:
class UserFactory(factory.django.DjangoModelFactory):
    username = factory.Sequence(lambda n: "user-{0}".format(n))
    email = factory.Sequence(lambda n: "user-{0}@example.com".format(n))
    password = factory.PostGenerationMethodCall("set_password", "password")

    class Meta:
        model = get_user_model()
        django_get_or_create = ("username",)



class PageFactory(factory.django.DjangoModelFactory):
    class Meta:
        abstract = True

    @classmethod
    def _create(cls, model_class, *args, **kwargs):

        try:
            parent = kwargs.pop("parent")
        except KeyError:
            # no parent, appending page to root
            parent = Page.get_first_root_node()
            site = Site.objects.first()
            site.root_page = parent
            site.save()


        page = model_class(*args, **kwargs)
        parent.add_child(instance=page)

        return page


class BlogFactory(PageFactory):
    author = None
    title = factory.Sequence(lambda n: "blog-{0}".format(n))
    slug = factory.Sequence(lambda n: "blog-{0}".format(n))

    class Meta:
        model = Blog
        django_get_or_create = ("slug",)


class PostFactory(PageFactory):
    class Meta:
        model = Post
        django_get_or_create = ("slug",)

In [7]:
Blog.objects.all().delete()
Post.objects.all().delete()

In [8]:
user = UserFactory()
user._password = "password"

blog = BlogFactory(owner=user, title="testblog", slug="testblog", description="foobar baz")
post = PostFactory(
    owner=blog.owner,
    parent=blog,
    title="test entry",
    slug="test-entry",
    pub_date=timezone.now(),
)

In [12]:
post.get_url(None)

'/testblog/test-entry/'

In [10]:
post.blog.description

'foobar baz'

In [20]:
site = (Site.objects.select_related('root_page', 'root_page__locale')
             .order_by('-root_page__url_path', '-is_default_site', 'hostname')
             .first()) 

In [21]:
site.root_page.url_path

'/'

In [22]:
print(blog.url)

possible sites:  [(1, '/', 'http://localhost', 'en')]
/testblog/


In [23]:
blog.url

possible sites:  [(1, '/', 'http://localhost', 'en')]


'/testblog/'

In [24]:
Site.get_site_root_paths()

[SiteRootPath(site_id=1, root_path='/', root_url='http://localhost', language_code='en')]

In [25]:
blog.get_parent()

<Page: Root>

In [26]:
site.root_page

<Page: Root>

In [27]:
Site.root_page

In [28]:
blog._get_site_root_paths()

[SiteRootPath(site_id=1, root_path='/', root_url='http://localhost', language_code='en')]

In [29]:
blog._wagtail_cached_site_root_paths

[SiteRootPath(site_id=1, root_path='/', root_url='http://localhost', language_code='en')]

In [30]:
blog.set_url_path(blog.get_parent())

'/testblog/'

In [31]:
blog.get_parent().url_path

'/'

In [32]:
Page.objects.all()

<PageQuerySet [<Page: Root>, <Page: Welcome to your new Wagtail site!>, <Page: testblog>, <Page: test entry>]>

In [33]:
post.url

possible sites:  [(1, '/', 'http://localhost', 'en')]


'/testblog/test-entry/'

In [5]:
post.save()

In [7]:
post.blog.url_path

'/testblog/'

In [5]:
post.url

possible sites:  []


In [6]:
post.set_url_path(post.blog)

'/testblog/test-entry/'

In [10]:
post.url_path

'/testblog/test-entry/'

In [11]:
post.url

possible sites:  []


In [13]:
blog.set_url_path(blog.get_parent())

'/testblog/'

In [8]:
post._get_site_root_paths(None)

[SiteRootPath(site_id=1, root_path='/home/', root_url='http://localhost', language_code='en')]

In [9]:
[(pk, path, url, language_code) for pk, path, url, language_code in post._get_site_root_paths(None)]

[(1, '/home/', 'http://localhost', 'en')]

In [13]:
post.url_path = "/home/foo-4/"

In [14]:
[(pk, path, url, language_code)
            for pk, path, url, language_code in post._get_site_root_paths(None)
            if post.url_path.startswith(path)]

[(1, '/home/', 'http://localhost', 'en')]

In [16]:
post.full_url

possible sites:  [(1, '/home/', 'http://localhost', 'en')]


'http://localhost/foo-4/'

In [5]:
post._get_site_root_paths()

[SiteRootPath(site_id=1, root_path='/home/', root_url='http://localhost', language_code='en')]

In [6]:
post.url_path

'/foo-3/'

In [11]:
post.url_path

'/home/mein-kleines-blog/first-post/'

In [9]:
post.get_parent()

<Page: Mein kleines Blog>

In [10]:
post.save()

In [16]:
print(post.url)

None


In [17]:
post.url_path

'/foo/'

In [18]:
help(post.url)

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [7]:
post = Post.objects.first()

In [21]:
help(post.url)

No Python documentation found for '/mein-kleines-blog/first-post/'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.



In [22]:
post.full_url

'http://localhost/mein-kleines-blog/first-post/'

In [28]:
post.get_url_parts()

(1, 'http://localhost', '/mein-kleines-blog/first-post/')

In [8]:
post.url_path

'/home/mein-kleines-blog/first-post/'

In [12]:
post.allowed_subpage_models()

[wagtail.core.models.Page,
 cast.models.HomePage,
 cast.models.BlogIndexPage,
 cast.models.Blog,
 cast.models.Post]

In [13]:
post.blog.allowed_subpage_models()

[wagtail.core.models.Page,
 cast.models.HomePage,
 cast.models.BlogIndexPage,
 cast.models.Blog,
 cast.models.Post]

In [17]:
help(post.set_url_path)

Help on method set_url_path in module wagtail.core.models:

set_url_path(parent) method of cast.models.Post instance
    Populate the url_path field based on this page's slug and the specified parent page.
    (We pass a parent in here, rather than retrieving it via get_parent, so that we can give
    new unsaved pages a meaningful URL when previewing them; at that point the page has not
    been assigned a position in the tree, as far as treebeard is concerned.

